<p style="text-align: center;" ><font size="7" >Data download</font></p>
<hr />

# Preparation

## Dependancies

In [13]:
import pandas as pd
from collections import defaultdict
import os
import shutil as sh
import urllib
import tarfile

from ipywidgets import interact, interactive, fixed, interact_manual, IntProgress
import ipywidgets as widgets
from IPython.display import display
import gzip

#Pandarallel works only on linux and mac
try:
    from pandarallel import pandarallel
    pandarallel.initialize(nb_workers=8,progress_bar=True)
    PARRALEL = True
except:
    PARRALEL = False

from tqdm.notebook import tnrange, tqdm
tqdm.pandas() #activate tqdm progressbar for pandas apply

## import folder configuration

You can change folders setting in the "00-SETUP.ipynb" notebook

In [14]:
%run "./00-SETUP.ipynb"
UPDATE = False

# Donwload files 

## Downloading the CATH domain list

In [15]:
domfile = 'cath-domain-list.txt'

url = "ftp://orengoftp.biochem.ucl.ac.uk/cath/releases/all-releases/v4_2_0/cath-classification-data/cath-domain-list-v4_2_0.txt"
destination = CATHFOLDER+domfile
if not os.path.isfile(destination) or UPDATE: 
    urllib.request.urlretrieve(url, destination)
    

colomnDomFile = [
    'Domain','Class','Architecture','Topology','Homologous','S35','S60','S95','S100','S100Count','DomSize','resolution',
]

## Download Correspondance between Uniprot and PDB code

In [16]:
url="ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/flatfiles/csv/pdb_chain_uniprot.csv.gz"
destination = CATHFOLDER+"pdb_chain_uniprot.csv.gz" 
if not os.path.isfile(destination) or UPDATE: 
    urllib.request.urlretrieve(url, destination)
    
with gzip.open(destination, 'rb') as f:
    file = f.read()
    
    with open(destination.split('.gz')[0], 'wb') as output:
        output.write(file)
    #.write(destination.split('.gz')[0])

os.remove(destination)

## Download PROSITE files


In [17]:
url="ftp://ftp.expasy.org/databases/prosite/prosite_alignments.tar.gz"
destination = PROSITEFOLDER+"prosite_alignments.tar.gz" 

if UPDATE:
    if os.path.exists(PROSITEFOLDER+"msa"):
        import shutil
        shutil.rmtree(PROSITEFOLDER+"msa/")
    else:
        os.makedirs(PROSITEFOLDER+"msa/")

if not os.path.exists(PROSITEFOLDER+"msa") or UPDATE: 
    urllib.request.urlretrieve(url, destination)
    tf = tarfile.open(destination)
    tf.extractall(PROSITEFOLDER)
    tf.close()
    os.rename(PROSITEFOLDER+"prosite_alignments",PROSITEFOLDER+"msa")
    os.remove(destination)


    



## Download CATH PDB files

Reading cath domain list

In [18]:
cathDomains = pd.read_csv(CATHFOLDER+domfile,comment='#', sep=r"\s+", header=None)
cathDomains.columns = colomnDomFile
if PARRALEL:
    cathDomains['Superfamily'] = cathDomains.parallel_apply(lambda x: f"{x.Class}.{x.Architecture}.{x.Topology}.{x.Homologous}", axis=1)
else:
    cathDomains['Superfamily'] = cathDomains.progress_apply(lambda x: f"{x.Class}.{x.Architecture}.{x.Topology}.{x.Homologous}", axis=1)

  0%|          | 0/434857 [00:00<?, ?it/s]

Creating the superfamily

In [19]:
cathSuperFamily = pd.DataFrame()
cathSuperFamily['Superfamily'] = cathDomains.Superfamily
cathSuperFamily['Domain'] = cathDomains.Domain

Creating a dictionary with the superfamily as key and list of cathdomain (pdb format) as value

In [20]:
cathDomainsPerSuperFamily = defaultdict(list)
#do not parralel this one
_ = cathSuperFamily.progress_apply(lambda x: cathDomainsPerSuperFamily[x.Superfamily].append(x.Domain), axis=1)

  0%|          | 0/434857 [00:00<?, ?it/s]

## Gather data for ALL domains

In [21]:
def download_dom(dom, folder):
    url = "http://www.cathdb.info/version/"+CATHVERSION+"/api/rest/id/"+dom+".pdb"
    destination = folder+dom+'.pdb'
    if not os.path.isfile(destination): 
        urllib.request.urlretrieve(url, destination)
    #progressbar.value += 1

def fetch_dom_for_superfamily(superfamily, cathDomainsPerSuperFamily, domName):
    print(f">Working with {domName} domain")
    global CATHFOLDER
    folder = CATHFOLDER+'domains/'+domName+'/raw/'
    if not os.path.exists(folder):
        os.makedirs(folder)
    if not os.path.exists(CATHFOLDER+'domains/'+domName+'/cleaned/'):
        os.makedirs(CATHFOLDER+'domains/'+domName+'/cleaned/')

    domlist = cathDomainsPerSuperFamily[superfamily]
    
    if PARRALEL:
        pd.Series(domlist).parallel_apply(lambda x: download_dom(x, folder))
    else:
        print(domlist)
        pd.Series(domlist).progress_apply(lambda x: download_dom(x, folder))


        
        
for superfamily,domain in SUPERFAMILY.items():
    fetch_dom_for_superfamily(superfamily, cathDomainsPerSuperFamily, domain)
    

>Working with PH domain
['1unqA00', '1unpA00', '2uvmA00', '1unrA00', '1h10A00', '2uzrA00', '2uzsA00', '2x18E00', '2x18H00', '2x18C00', '2x18D00', '2x18G00', '2x18B00', '2x18A00', '2x18F00', '3aj4A00', '3aj4B00', '3viaA00', '3viaB00', '2dhiA00', '2d9vA01', '1xodB00', '1xodA00', '1tj6A00', '1tj6B00', '3syxA00', '2jp2A00', '3pp2A00', '3pp2B00', '2p0hA00', '2p0fA00', '2p0dA00', '4wsfA00', '2y8gB00', '2y8gA00', '2y8fC00', '2y8fB00', '2y8fA00', '2y8fD00', '2crfA01', '4a6hA00', '4a6hB00', '4a6hC00', '4a6hD00', '4a5kB00', '4a5kA00', '4a5kD00', '4a5kC00', '4a6kA00', '4a6kB00', '4a6fA00', '4a6kC00', '4a6kD00', '3nsuA00', '3nsuB00', '4a6fB00', '1fgyA00', '1u2bA00', '1fgzA00', '1u29A00', '1u27A00', '1fhwA00', '1fhwB00', '2r09B03', '2r09A03', '2r0dB03', '2r0dA03', '4kaxB00', '1fhxA00', '1fhxB00', '1upqA00', '1uprA00', '1eazA00', '2dtcA00', '2dtcB00', '4tyzA00', '4tyzB00', '3f5rA00', '4khbB01', '4khbF01', '4khbH01', '4khbD01', '4my6B00', '4my6A00', '1evhA00', '2iybA00', '2iybB00', '2iybC00', '2iybD0

  0%|          | 0/818 [00:00<?, ?it/s]

>Working with C2 domain
['4weeA01', '3f00A01', '3f01A01', '1bynA00', '2k45A00', '2k4aA00', '2k8mA00', '2k8mD00', '2ki6A00', '2ki6F00', '3f04A00', '3f05A00', '1rsyA00', '5cchF01', '2r83A01', '2r83B01', '5ccgE01', '5ccgK01', '5kj7E01', '5kj7K01', '5ccgF01', '5kj7F01', '5cchE01', '5h4yA00', '5h4zA00', '5h4zB00', '1dqvA01', '3hn8A01', '3hn8B01', '3hn8C01', '3kwuA00', '3kwtA00', '2cm5A00', '2cm6B00', '2cm6A00', '3rpbA00', '4ldcA00', '3n5aA00', '1tjxA00', '1uowA00', '1tjmA00', '1uovA00', '4v11A00', '2r83A02', '2r83B02', '2yoaA00', '2yoaB00', '5cchF02', '5ccgF02', '5kj7F02', '5kj7E02', '5kj7K02', '5ccgE02', '5ccgK02', '5cchE02', '1k5wA00', '2lhaA00', '2n1tE00', '5ccjA00', '5ccjC00', '5ccjB00', '5ccjD00', '1w15A00', '1w16A00', '1dqvA02', '3hn8A02', '3hn8B02', '3hn8C02', '3twyA00', '4l1lA00', '3rdjA00', '3gpeA00', '1dsyA00', '2nceA00', '3rdlA00', '4dnlA00', '2uzpA00', '2uzpB00', '2uzpC00', '1a25A00', '1a25B00', '3pfqA02', '2cjtC00', '2cjtD00', '2cjtA00', '2cjtB00', '2cjsA01', '2cjsB01', '1gmiA0

  0%|          | 0/346 [00:00<?, ?it/s]

>Working with C1 domain
['2orwB02', '2orwA02', '2qq0A02', '2qq0B02', '2qqeA02', '2qqeB02', '2qpoA02', '2qpoB02', '2qpoC02', '2qpoD02', '3uejA01', '3uejB01', '3ugiA01', '3ugiB01', '3uglA01', '3uglB01', '1ptqA00', '1ptrA00', '3ueyA01', '3ueyB01', '3uffA01', '3uffB01', '3ugdA01', '3ugdB01', '4fkdA01', '2enzA01', '2eliA01', '2vrwB03', '3e2iA02', '1xx6A02', '1xx6B02', '2j9rA02', '2ja1A02', '1w4rA02', '1w4rB02', '1w4rC02', '1w4rD02', '1w4rE02', '1w4rF02', '1w4rG02', '1w4rH02', '2orvA02', '2orvB02', '1xbtF02', '1xbtA02', '1xbtB02', '1xbtC02', '1xbtD02', '1xbtE02', '1xbtG02', '1xbtH02', '2wvjB02', '2wvjA02', '2wvjC02', '2wvjD02', '2wvjE02', '2wvjF02', '2wvjG02', '2wvjH02', '2j87A02', '2j87B02', '2j87C02', '2j87D02', '5fuwA02', '5fuwB02', '5fuxA02', '5fuxB02', '5fuvA02', '5fuvB02', '5fuyE02', '5fuyA02', '5fuyC02', '5fuyF02', '5fuyB02', '5fuyD02', '4uxhB02', '4uxhA02', '4uxiA02', '4uxiB02', '4uxjA02', '4uxjF02', '4uxjB02', '4uxjC02', '4uxjD02', '4uxjE02', '4uxjG02', '4uxjH02', '4b6dB00', '4b6dE0

...

  0%|          | 0/150 [00:00<?, ?it/s]

>Working with PLA domain
['1mc2A00', '1mg6A00', '3i03A00', '3i3iA00', '3cxiA00', '3cxiB00', '3hzdA00', '3hzdB00', '4k06A00', '4k06B00', '3i3hA00', '3i3hB00', '3hzwA00', '3hzwB00', '3iq3A00', '3iq3B00', '2h8iA00', '2h8iB00', '2q2jA00', '2q2jB00', '3cylA00', '3cylB00', '4yu7A00', '4yu7B00', '3qnlA00', '3qnlB00', '4yz7A00', '4yz7B00', '2ok9A00', '2ok9B00', '1qllA00', '1qllB00', '4wtbA00', '4wtbB00', '1pa0A00', '1pa0B00', '3mlmA00', '3mlmB00', '1pc9A00', '1pc9B00', '1s8iA00', '1s8hA00', '1s8gA00', '1ppaA00', '1y4lA00', '1y4lB00', '1clpA00', '1clpB00', '4k09A00', '4k09B00', '1xxsA00', '1xxsB00', '4yv5A00', '4yv5B00', '4kf3A00', '4kf3B00', '2ph4A00', '2ph4B00', '2aozA00', '3t0rA00', '3t0rB00', '3t0rC00', '3t0rD00', '4dcfA00', '4dcfB00', '4dcfC00', '4dcfD00', '1godA00', '1zlbA00', '1zl7A00', '1umvX00', '1z76A00', '1z76B00', '1u73A00', '1u73B00', '4rfpA00', '4rfpB00', '4h0qA00', '4h0qB00', '1vapA00', '1vapB00', '1m8rA00', '1m8sA00', '1bk9A00', '1psjA00', '1pp2L00', '1pp2R00', '1ijlA00', '1ijlB

  0%|          | 0/505 [00:00<?, ?it/s]

>Working with ANNEXIN domain
['2ii2A04', '2iicA04', '2iicB04', '2iicC04', '2iicD04', '2ii2A01', '2iicA01', '2iicB01', '2iicC01', '2iicD01', '2zhjA02', '2zhiA02', '1i4aA02', '1annA02', '1aowA02', '2zocA02', '2zocB02', '1yiiA02', '1alaA02', '1yj0A02', '2ie7A02', '2ie6A02', '2ranA02', '1a8aA02', '1a8bA02', '1g5nA02', '1bcyA02', '1bc0A02', '1bcwA02', '1n41A02', '1bczA02', '2h0mA02', '2h0kA02', '2h0kB02', '1n44A02', '1bc3A02', '1bc1A02', '1n42A02', '2h0lA02', '1anxA02', '1anxB02', '1anxC02', '1hvdA02', '1hvfA02', '1avhA02', '1avhB02', '1avrA02', '1hveA02', '1anwA02', '1anwB02', '1hakA02', '1hakB02', '1hvgA02', '2xo3A02', '1savA02', '2xo2A02', '1w3wA02', '1w45A02', '1w45B02', '1m9iA02', '1avcA02', '2zhjA01', '2zhiA01', '1i4aA01', '1annA01', '1aowA01', '2zocA01', '2zocB01', '1dm5A01', '1dm5B01', '1dm5C01', '1dm5D01', '1dm5E01', '1dm5F01', '1aeiA01', '1aeiB01', '1aeiC01', '1aeiD01', '1aeiE01', '1aeiF01', '1dk5B01', '1dk5A01', '2hyvA04', '1w7bA04', '2hyuA04', '2hywA04', '2hywB04', '1xjlA04', '1

  0%|          | 0/405 [00:00<?, ?it/s]

**Now the next notebook can be runed** 😀